<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Demo_Weights_and_Biases_report_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Installation and import

In [1]:
!pip install wandb
!python -m pip install -q dlomix==0.0.4
import wandb
import wandb.apis.reports as wr
from wandb.keras import WandbCallback
import dlomix
from dlomix.data import RetentionTimeDataset
from dlomix.models import RetentionTimePredictor
import tensorflow as tf
from dlomix.eval import TimeDeltaMetric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=cbd3115ce43e70ccc11b9c3e88b094cb130c043519eb222454e154993d6fd068
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
  Preparing metadata (setup.py) ... done


wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


# 1. Log sample data

In [2]:
# project name for weights and biases
project_name = 'demo_wandb_report_api'

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'
BATCH_SIZE = 64

rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, batch_size=BATCH_SIZE, val_ratio=0.2, test=False)
model = RetentionTimePredictor(seq_length=30)
wandb.init(project=project_name, name='sample_data')

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile model
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mean_absolute_error'])
# train model
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=15,callbacks=[WandbCallback(save_model=False)])
# finish run
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 1/15
424/424 [==============================] - 19s 14ms/step - loss: 2055.6130 - mean_absolute_error: 35.5567 - val_loss: 600.1982 - val_mean_absolute_error: 19.9488
Epoch 2/15
424/424 [==============================] - 6s 15ms/step - loss: 365.5186 - mean_absolute_error: 14.9730 - val_loss: 242.0835 - val_mean_absolute_error: 11.8827
Epoch 3/15
424/424 [==============================] - 6s 14ms/step - loss: 205.3772 - mean_absolute_error: 10.8298 - val_loss: 179.4427 - val_mean_absolute_error: 10.0089
Epoch 4/15
424/424 [==============================] - 5s 13ms/step - loss: 169.0285 - mean_absolute_error: 9.6862 - val_loss: 160.8760 - val_mean_absolute_error: 9.3522
Epoch 5/15
424/424 [==============================] - 7s 16ms/step - loss: 154.7374 - mean_absolute_error: 9.1981 - val_loss: 151.0702 - val_mean_absolute_error: 8.9847
Epoch 6/15
424/424 [==============================] - 6s 14ms/step - loss: 145.9552 - mean_absolute_error: 8.8880 - val_loss: 143.6206 - val_mean_a

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_mean_absolute_error,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁
best_epoch,14
best_val_loss,105.09791
epoch,14
loss,101.23648
mean_absolute_error,7.16074
val_loss,105.09791


In [3]:
# Create report
report = wr.Report(
    project = 'demo_wandb_report_api',
    title = 'Demo report',
    description = 'A quick showcase of wandbs report api.'
)
# Save report
report.save()

Report(project='demo_wandb_report_api', entity='master_praktikum', title='Demo report', description='A quick showcase of wandbs report api.', width='readable')

In [4]:
# Adding content to the report
report.blocks = [
    wr.TableOfContents(),
    wr.H1("Classic latin text"),
    wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo?")
]
# Save report
report.save()

Report(project='demo_wandb_report_api', entity='master_praktikum', title='Demo report', description='A quick showcase of wandbs report api.', width='readable', blocks=[TableOfContents(), H1(text=['Classic latin text']), P(text=['Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo?'])])

In [6]:
# Create PanelGrid block
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(
        entity = wandb.apis.PublicApi().default_entity,
        project = 'demo_wandb_report_api'
        )
    ],
    panels=[
        wr.LinePlot(x='Step', y=['val_loss']),
        wr.BarPlot(metrics=['loss'])
    ]
)
# Add block to report and save
report.blocks = report.blocks + [wr.H1("Panel Grid Example"), pg]
report.save()

Report(project='demo_wandb_report_api', entity='master_praktikum', title='Demo report', description='A quick showcase of wandbs report api.', width='readable', blocks=[TableOfContents(), H1(text=['Classic latin text']), P(text=['Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo?']), H1(text=['Panel Grid Example']), PanelGrid(runsets=[Runset(entity='master_praktikum', project='demo_wandb_report_api', name='Run set', query='', filters={'$or': [{'$and': []}]}, order=['-CreatedTimestamp'])], panels=[LinePlot(x='Step', y=['val_acc']), BarPlot(metrics=['acc'], orientation='h')]), H1(text=['Panel Grid Example']), PanelGrid(runsets=[Runset(entity='master_praktikum', project='demo_wandb_report_api', name='Run set', query='', filters={'$or': [{'$and': []}]}, order=['-CreatedTimestamp'])], panels=[LinePlot(x='Step', y=['val_loss']), BarPlot(metrics=['loss'], orientation='h')])])